<h2 align="center" style="font-size: 18px;">Задание №1</h2>
<h2 align="center" style="font-size: 18px;">тема: «Задача классификации в машинном обучении»</h2>

## Описание задач

------------

Задания сформулированы для работы в MATLAB и выполнены с помощью инструментов Python.

-------------

I. Создать файлы признаков Харалика для изображений в папках YES и NO при следующих параметрах матрицы а смежности уровней серого:   

а) G=16, d=2  б) G=128, d=4 

II. Используя все классификаторы MATLAB провести классификацию признаков Харалика для изображений в папках YES и NO с параметрами из пунктов  а) и б). 
    
Использовать три значения параметра кросс-валидации: а) К=3; б) К=4; в) К=5.

Определить по три модели классификации, имеющих наибольшие средние значения точности, чувствительности и специфичности (усреднять для различных значений К)

## Процесс выполнения


Для выполнения задач использовался датасет **Chest X-Ray Images (Pneumonia)**, содержащий рентгеновские изображения лёгких: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia. 

Были взяты папки NORMAL (324 файла .jpeg) - изображения без паталогий, PNEUMONIA (390 файлов .jpeg) - изображения с заболеванием.

Из каждого изображения будут извлечны характеризующие текстурные признаки для подачи на вход классификаторам.

Извлечение признаков для одного изображения проходит этапы:
1. Представление изображения в виде матрицы яркости в градациях серого (диапазон 0-255).
2. Квантование яркости на G уровней
3. Вычисление по заданным параметрам матриц GLCM по 4-м направлениям (0°, 45°, 90°, 135°)
4. Извлечение из матриц GLCM признаков Харалика

### Загрузка данных и создание датафреймов


In [1]:
import warnings
warnings.filterwarnings('ignore')


import os
import cv2
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

In [2]:
# Путь к папке с изображениями без заболеваний
normal_path = '/home/katya/Магистратура КФУ/3 семестр/Машинное обучение в медицине/task_1/NORMAL/'

# Считывание изображений в оттенках серого
if not os.path.exists(normal_path):
    print(f"Ошибка: Папка не найдена: {normal_path}")
else:
    normal_data = []
    normal_files = [f for f in os.listdir(normal_path) if f.lower().endswith('.jpeg')]
    
    print(f"Найдено {len(normal_files)} изображений без патологий")
    
    for i, filename in enumerate(normal_files):
        file_path = os.path.join(normal_path, filename)
        img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
        
        if img is not None:
            file_name_without_ext = Path(filename).stem
            normal_data.append({
                'image': img,
                'label': 0
            })
        else:
            print(f"Предупреждение: не удалось загрузить {filename}")

Найдено 234 изображений без патологий


In [3]:
# Путь к папке с изображениями с пневмонией
pneumonia_path = '/home/katya/Магистратура КФУ/3 семестр/Машинное обучение в медицине/task_1/PNEUMONIA/'

# Считывание изображений в оттенках серого
if not os.path.exists(pneumonia_path):
    print(f"Ошибка: Папка не найдена: {pneumonia_path}")
else:
    pneumonia_data = []
    pneumonia_files = [f for f in os.listdir(pneumonia_path) if f.lower().endswith('.jpeg')]
    
    print(f"Найдено {len(pneumonia_files)} изображений с пневмонией")
    
    for i, filename in enumerate(pneumonia_files):
        file_path = os.path.join(pneumonia_path, filename)
        img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
        
        if img is not None:
            file_name_without_ext = Path(filename).stem
            pneumonia_data.append({
                'image': img,
                'label': 1
            })
        else:
            print(f"Предупреждение: не удалось загрузить {filename}")

Найдено 390 изображений с пневмонией


In [4]:
# Создание датафреймов
normal_df = pd.DataFrame(normal_data)
pneumonia_df = pd.DataFrame(pneumonia_data)

In [5]:
normal_df.head()

,image,label
0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0
1,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0
2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0
3,"[[1, 14, 15, 11, 15, 14, 11, 15, 14, 14, 14, 1...",0
4,"[[1, 26, 22, 23, 26, 21, 26, 22, 22, 22, 22, 2...",0


In [6]:
pneumonia_df.head()

,image,label
0,"[[18, 19, 21, 24, 26, 29, 31, 32, 35, 37, 39, ...",1
1,"[[19, 19, 19, 20, 21, 22, 24, 25, 28, 30, 34, ...",1
2,"[[15, 15, 15, 16, 16, 17, 17, 17, 20, 20, 20, ...",1
3,"[[186, 189, 191, 189, 185, 182, 181, 182, 187,...",1
4,"[[37, 37, 37, 37, 37, 37, 37, 37, 39, 39, 39, ...",1


In [7]:
normal_df.shape

(234, 2)

In [8]:
pneumonia_df.shape

(390, 2)

Визуализируем первые пять изображений без патологий.

In [9]:
# Визуалиация рентгеновских снимков
#fig, axes = plt.subplots(1, 5, figsize=(20, 4))
#for i in range(min(5, len(normal_df))):
#    axes[i].imshow(normal_df.iloc[i, 1], cmap='gray')
#    axes[i].set_title(str(normal_df.iloc[i, 0]))
#    axes[i].axis('off')

#plt.tight_layout()
#plt.show()

### I. Извлечение признаков Харалика

Текстурные признаки Харалика — набор из 14 статистических признаков, предложенных Р. Хараликом для количественного описания текстуры изображения. Все признаки вычисляются на основе матрицы смежностей уровней серого (GLCM, Матрица Харалика), которая показывает, как часто разные комбинации яркостей пикселей встречаются на заданном расстоянии и направлении.

В библиотеке **scikit-image** есть функции:
- `graycomatrix()`  - для автоматического вычисления матрицы GLCM;
-  `graycoprops()` - для расчёта 6-ти признаков Харалика: contrast, dissimilarity, homogeneity, energy, correlation, ASM.
Остальные признаки можно вычислить отдельными функциями.

Напишем программу, которая будет для каждого изображения вычислять GLCM отдельно по каждому из 4-х направлений и затем по этим матрицам вычислять признаки.

Поля создаваемых таблиц будут содержать: имя файла изображения, все названия интересуемых признаков, метка класса YES или NO.

Всего признаков будет 14 * 2 * 4 = 112 (14 признаков в двух вариантах параметрах по четырём направлениям)

#### Функции для признаков, которых нет в graycoprops

In [10]:
def maximum_probability(glcm):
    num_angles = glcm.shape[3]
    result = []
    for angle_idx in range(num_angles):
        matrix = glcm[:, :, 0, angle_idx]
        max_prob = matrix.max() / np.sum(matrix)
        result.append(max_prob)
    return np.array(result)

def entropy(glcm):
    eps = 1e-10
    num_angles = glcm.shape[3]
    result = []
    for angle_idx in range(num_angles):
        matrix = glcm[:, :, 0, angle_idx]
        P = matrix / np.sum(matrix)
        ent = -np.sum(P * np.log2(P + eps))
        result.append(ent)
    return np.array(result)

def variance(glcm):                       
    num_angles = glcm.shape[3]
    result = []
    for angle_idx in range(num_angles):
        matrix = glcm[:, :, 0, angle_idx]
        N = np.sum(matrix)
        gray_values = np.arange(matrix.shape[0])[:, None]        
        mu = np.sum(gray_values * matrix) / N
        var = np.sum((gray_values - mu) ** 2 * matrix) / N
        result.append(var) 
    return np.array(result)

def sum_average(glcm):
    num_angles = glcm.shape[3]
    result = []
    for angle_idx in range(num_angles):
        matrix = glcm[:, :, 0, angle_idx]
        G = matrix.shape[0]
        N = np.sum(matrix)      
        S = np.zeros(2*G - 1)
        for i in range(G):
            for j in range(G):
                S[i + j] += matrix[i, j]
        
        k_vals = np.arange(len(S))
        sum_avg = np.sum(k_vals * S) / N
        result.append(sum_avg)
    
    return np.array(result)

def sum_variance(glcm):
    num_angles = glcm.shape[3]
    result = []
    for angle_idx in range(num_angles):
        matrix = glcm[:, :, 0, angle_idx]
        G = matrix.shape[0]
        N = np.sum(matrix)
        S = np.zeros(2*G - 1)
        for i in range(G):
            for j in range(G):
                S[i + j] += matrix[i, j]
        sum_probs = S / N
        k_vals = np.arange(len(S))      
        sum_avg = np.sum(k_vals * sum_probs)
        sum_var = np.sum((k_vals - sum_avg) ** 2 * S) / N
        result.append(sum_var)
    
    return np.array(result)

def sum_entropy(glcm):
    eps = 1e-10
    num_angles = glcm.shape[3]
    result = []
    for angle_idx in range(num_angles):
        matrix = glcm[:, :, 0, angle_idx]
        G = matrix.shape[0]
        N = np.sum(matrix)
        S = np.zeros(2*G - 1)
        for i in range(G):
            for j in range(G):
                S[i + j] += matrix[i, j]
        sum_probs = S / N
        mask = sum_probs > 0
        sum_ent = -np.sum(sum_probs[mask] * np.log2(sum_probs[mask] + eps))
        result.append(sum_ent)
    
    return np.array(result)

def difference_variance(glcm):
    num_angles = glcm.shape[3]
    result = []
    for angle_idx in range(num_angles):
        matrix = glcm[:, :, 0, angle_idx]
        G = matrix.shape[0]
        N = np.sum(matrix)
        D = np.zeros(G)
        for i in range(G):
            for j in range(G):
                D[abs(i - j)] += matrix[i, j]
        diff_probs = D / N
        k_vals = np.arange(G)
        diff_avg = np.sum(k_vals * diff_probs)
        diff_var = np.sum((k_vals - diff_avg) ** 2 * D) / N
        result.append(diff_var)
    return np.array(result)

def difference_entropy(glcm):
    eps = 1e-10
    num_angles = glcm.shape[3]
    result = []
    for angle_idx in range(num_angles):
        matrix = glcm[:, :, 0, angle_idx]
        G = matrix.shape[0]
        N = np.sum(matrix)
        D = np.zeros(G)
        for i in range(G):
            for j in range(G):
                D[abs(i - j)] += matrix[i, j]
        diff_probs = D / N
        mask = diff_probs > 0
        diff_ent = -np.sum(diff_probs[mask] * np.log2(diff_probs[mask] + eps))
        result.append(diff_ent)
    
    return np.array(result)

#### Формирование матриц GLCM cо всеми наборами параметров

Всего: 2 набора по 4-м направлениям = 8 матриц GLCM. 

In [11]:
from skimage.feature import graycomatrix, graycoprops

In [12]:
# Функции для квантования изображения и автоматического вычисления GLCM. На выходе дают массив из 4-х матриц

def create_glcm_a(img_array):
    # Квантование для 16 уровней
    img_min = img_array.min()
    img_max = img_array.max()
    if img_max > img_min:
        img_quantized = ((img_array - img_min) / (img_max - img_min) * 15).astype(np.uint8)
    else:
        img_quantized = np.zeros_like(img_array, dtype=np.uint8)
    
    # Создание GLCM с параметрами a
    return graycomatrix(img_quantized, distances=[2], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], levels=16, symmetric=True)

def create_glcm_b(img_array):
    # Квантование для 128 уровней
    img_min = img_array.min()
    img_max = img_array.max()
    if img_max > img_min:
        img_quantized = ((img_array - img_min) / (img_max - img_min) * 127).astype(np.uint8)
    else:
        img_quantized = np.zeros_like(img_array, dtype=np.uint8)
    
    # Создание GLCM с параметрами b
    return graycomatrix(img_quantized, distances=[4], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], levels=128, symmetric=True)

In [13]:
# Применение функций к столбцу с матрицами изображений, создание двух новых столбцов с массивами матриц GLCM
pneumonia_df['GLCM_a'] = pneumonia_df['image'].apply(create_glcm_a)
pneumonia_df['GLCM_b'] = pneumonia_df['image'].apply(create_glcm_b)
normal_df['GLCM_a'] = normal_df['image'].apply(create_glcm_a)
normal_df['GLCM_b'] = normal_df['image'].apply(create_glcm_b)
print(pneumonia_df.columns.tolist())


['image', 'label', 'GLCM_a', 'GLCM_b']


In [14]:
# Разделение массивов матриц по отдельным столбцам

pneumonia_df['GLCM_a_0'] = pneumonia_df['GLCM_a'].apply(lambda x: x[:, :, 0, 0])
pneumonia_df['GLCM_a_45'] = pneumonia_df['GLCM_a'].apply(lambda x: x[:, :, 0, 1])
pneumonia_df['GLCM_a_90'] = pneumonia_df['GLCM_a'].apply(lambda x: x[:, :, 0, 2])
pneumonia_df['GLCM_a_135'] = pneumonia_df['GLCM_a'].apply(lambda x: x[:, :, 0, 3])

normal_df['GLCM_a_0'] = normal_df['GLCM_a'].apply(lambda x: x[:, :, 0, 0]) 
normal_df['GLCM_a_45'] = normal_df['GLCM_a'].apply(lambda x: x[:, :, 0, 1]) 
normal_df['GLCM_a_90'] = normal_df['GLCM_a'].apply(lambda x: x[:, :, 0, 2]) 
normal_df['GLCM_a_135'] = normal_df['GLCM_a'].apply(lambda x: x[:, :, 0, 3]) 

pneumonia_df['GLCM_b_0'] = pneumonia_df['GLCM_b'].apply(lambda x: x[:, :, 0, 0]) 
pneumonia_df['GLCM_b_45'] = pneumonia_df['GLCM_b'].apply(lambda x: x[:, :, 0, 1])
pneumonia_df['GLCM_b_90'] = pneumonia_df['GLCM_b'].apply(lambda x: x[:, :, 0, 2])
pneumonia_df['GLCM_b_135'] = pneumonia_df['GLCM_b'].apply(lambda x: x[:, :, 0, 3])

normal_df['GLCM_b_0'] = normal_df['GLCM_b'].apply(lambda x: x[:, :, 0, 0])
normal_df['GLCM_b_45'] = normal_df['GLCM_b'].apply(lambda x: x[:, :, 0, 1])
normal_df['GLCM_b_90'] = normal_df['GLCM_b'].apply(lambda x: x[:, :, 0, 2]) 
normal_df['GLCM_b_135'] = normal_df['GLCM_b'].apply(lambda x: x[:, :, 0, 3])

print(f"Колонок в pneumonia_df: {len(pneumonia_df.columns)}")
print("Колонки с GLCM:", [col for col in pneumonia_df.columns if 'GLCM' in col])
print(f"Колонок в normal_df: {len(normal_df.columns)}")
print("Колонки с GLCM:", [col for col in normal_df.columns if 'GLCM' in col])

Колонок в pneumonia_df: 12
Колонки с GLCM: ['GLCM_a', 'GLCM_b', 'GLCM_a_0', 'GLCM_a_45', 'GLCM_a_90', 'GLCM_a_135', 'GLCM_b_0', 'GLCM_b_45', 'GLCM_b_90', 'GLCM_b_135']
Колонок в normal_df: 12
Колонки с GLCM: ['GLCM_a', 'GLCM_b', 'GLCM_a_0', 'GLCM_a_45', 'GLCM_a_90', 'GLCM_a_135', 'GLCM_b_0', 'GLCM_b_45', 'GLCM_b_90', 'GLCM_b_135']


In [15]:
# Удалим лишние столбцы
pneumonia_df = pneumonia_df.drop(columns=['image','GLCM_a', 'GLCM_b'])
normal_df = normal_df.drop(columns=['image','GLCM_a', 'GLCM_b'])

In [16]:
# Просмотр
pneumonia_df.head()

,label,GLCM_a_0,GLCM_a_45,GLCM_a_90,GLCM_a_135,GLCM_b_0,GLCM_b_45,GLCM_b_90,GLCM_b_135
0,1,"[[205112, 5337, 128, 75, 154, 24, 3, 3, 0, 0, ...","[[208312, 3717, 135, 129, 173, 35, 3, 1, 0, 0,...","[[210692, 3234, 125, 77, 109, 26, 2, 2, 1, 0, ...","[[208574, 3770, 95, 104, 142, 24, 4, 0, 1, 0, ...","[[123634, 4258, 1880, 1264, 515, 199, 157, 123...","[[126418, 3834, 1366, 812, 317, 171, 152, 133,...","[[135528, 1179, 443, 295, 265, 206, 192, 133, ...","[[126490, 3740, 1431, 867, 342, 169, 152, 135,..."
1,1,"[[227458, 3498, 74, 28, 22, 3, 0, 0, 0, 0, 0, ...","[[230116, 2028, 61, 26, 24, 4, 0, 0, 0, 0, 0, ...","[[232218, 1254, 45, 13, 18, 1, 0, 0, 0, 0, 0, ...","[[230072, 2115, 57, 21, 14, 2, 0, 0, 0, 0, 0, ...","[[162802, 3754, 1396, 406, 208, 161, 137, 121,...","[[164948, 3349, 905, 232, 162, 106, 98, 98, 63...","[[171932, 1106, 201, 89, 105, 86, 95, 64, 58, ...","[[164902, 3252, 968, 290, 183, 121, 91, 103, 6..."
2,1,"[[360236, 10119, 61, 5, 0, 0, 0, 0, 0, 0, 0, 0...","[[364558, 6450, 59, 7, 0, 0, 0, 0, 0, 0, 0, 0,...","[[364826, 6942, 79, 9, 0, 0, 0, 0, 0, 0, 0, 0,...","[[363484, 7534, 56, 6, 0, 0, 0, 0, 0, 0, 0, 0,...","[[101330, 6869, 1845, 1416, 1353, 1096, 1192, ...","[[104558, 6501, 1497, 1057, 963, 660, 560, 182...","[[109736, 3920, 1145, 723, 576, 378, 189, 98, ...","[[101678, 6717, 1784, 1379, 1462, 1188, 1138, ..."
3,1,"[[72006, 3593, 553, 31, 2, 4, 0, 1, 1, 6, 16, ...","[[72972, 3043, 103, 9, 3, 3, 2, 3, 2, 8, 26, 3...","[[73804, 2358, 90, 16, 9, 9, 3, 4, 2, 8, 24, 3...","[[73502, 2594, 55, 7, 3, 6, 1, 1, 2, 4, 22, 31...","[[3170, 1364, 1499, 707, 268, 168, 127, 131, 1...","[[2996, 1638, 1546, 584, 222, 202, 176, 167, 2...","[[6984, 1490, 590, 219, 154, 128, 99, 80, 84, ...","[[4446, 1838, 1093, 341, 186, 191, 174, 185, 2..."
4,1,"[[17864, 1680, 86, 20, 7, 2, 0, 0, 2, 0, 0, 0,...","[[18756, 917, 48, 34, 10, 4, 1, 1, 2, 0, 0, 0,...","[[19036, 840, 52, 22, 16, 14, 0, 1, 5, 0, 0, 0...","[[18546, 1142, 56, 30, 7, 5, 0, 0, 3, 0, 0, 0,...","[[126, 147, 323, 238, 108, 53, 44, 45, 51, 61,...","[[304, 162, 360, 191, 54, 67, 45, 53, 58, 86, ...","[[1146, 308, 151, 69, 46, 44, 19, 26, 14, 7, 7...","[[348, 237, 336, 154, 90, 69, 49, 53, 49, 61, ..."


In [17]:
# Просмотр
normal_df.head()

,label,GLCM_a_0,GLCM_a_45,GLCM_a_90,GLCM_a_135,GLCM_b_0,GLCM_b_45,GLCM_b_90,GLCM_b_135
0,0,"[[1137964, 16771, 370, 372, 661, 590, 248, 170...","[[1148998, 9878, 164, 201, 324, 308, 137, 83, ...","[[1158198, 4338, 36, 55, 68, 84, 88, 12, 0, 2,...","[[1148840, 9962, 165, 205, 321, 326, 147, 85, ...","[[914956, 2283, 609, 242, 143, 152, 284, 669, ...","[[920462, 2246, 600, 242, 146, 139, 237, 545, ...","[[939370, 2262, 703, 241, 117, 87, 80, 55, 56,...","[[920482, 2227, 597, 243, 146, 146, 244, 526, ..."
1,0,"[[478104, 4925, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[481730, 3101, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[483834, 2614, 12, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[481774, 3004, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[406546, 4525, 2725, 1083, 417, 163, 38, 25, ...","[[409360, 4163, 2063, 627, 224, 102, 31, 20, 1...","[[419522, 2423, 427, 79, 44, 45, 25, 23, 8, 10...","[[409664, 4066, 1905, 572, 174, 73, 27, 26, 15..."
2,0,"[[774090, 26663, 357, 430, 113, 0, 1, 8, 2233,...","[[790062, 16195, 218, 205, 68, 0, 1, 5, 1053, ...","[[803126, 8365, 113, 57, 3, 0, 1, 4, 3, 4, 0, ...","[[790112, 16073, 232, 209, 69, 0, 1, 6, 1064, ...","[[460208, 1934, 146, 34, 17, 22, 174, 254, 850...","[[466662, 1856, 157, 36, 17, 16, 178, 314, 779...","[[487780, 665, 120, 50, 24, 27, 17, 11, 20, 3,...","[[466566, 1834, 146, 35, 12, 19, 177, 319, 800..."
3,0,"[[866116, 5865, 178, 188, 144, 187, 172, 348, ...","[[868124, 4239, 228, 198, 135, 220, 162, 302, ...","[[872760, 4019, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[868266, 4044, 224, 198, 135, 220, 162, 302, ...","[[92, 252, 223, 405, 1102, 247, 193, 101, 18, ...","[[214, 262, 208, 535, 809, 526, 83, 19, 21, 13...","[[6264, 894, 270, 89, 55, 63, 22, 9, 12, 12, 5...","[[116, 288, 234, 545, 832, 516, 86, 9, 8, 7, 8..."
4,0,"[[121810, 18278, 378, 466, 389, 183, 182, 227,...","[[123990, 16091, 43, 521, 284, 221, 163, 214, ...","[[128354, 16826, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[[123732, 16338, 54, 522, 284, 221, 163, 213, ...","[[2, 14, 50, 43, 15, 2, 7, 11, 24, 210, 317, 1...","[[2, 29, 55, 30, 17, 14, 15, 111, 364, 222, 11...","[[4702, 658, 181, 36, 8, 8, 4, 1, 1, 1, 0, 7, ...","[[0, 19, 52, 32, 22, 8, 8, 95, 352, 215, 106, ..."


На этом шаге имеем 8 разных матриц GLCM, по которым вычислим 14 признаков Харалика.

In [18]:
# Функция для создания 14-ти колонок с признаками из одной с матрицами GLCM

def extract_haralick_features_from_column(df, glcm_column, params_suffix, angle_suffix):
    
    # Список 6 признаков из graycoprops
    builtin_features = ['contrast', 'dissimilarity', 'homogeneity', 
                       'energy', 'correlation', 'ASM']
    
    # Словарь функций для остальных признаков
    custom_functions = {
        'maximum_probability': maximum_probability,
        'entropy': entropy,
        'variance': variance,
        'sum_average': sum_average,
        'sum_variance': sum_variance,
        'sum_entropy': sum_entropy,
        'difference_variance': difference_variance,
        'difference_entropy': difference_entropy
    }
    
    for idx, glcm_matrix in enumerate(df[glcm_column]):
        if isinstance(glcm_matrix, list):
            if len(glcm_matrix) == 256:
                matrix = np.array(glcm_matrix).reshape(16, 16)
            elif len(glcm_matrix) == 16384:
                matrix = np.array(glcm_matrix).reshape(128, 128)
            else:
                matrix = np.array(glcm_matrix)
        else:
            matrix = glcm_matrix
        glcm_4d = matrix.reshape(matrix.shape[0], matrix.shape[1], 1, 1)
        
        # Встроенные в библиотеку признаки
        for feature_name in builtin_features:
            value = graycoprops(glcm_4d, feature_name)[0, 0]
            col_name = f"{feature_name}_{params_suffix}_{angle_suffix}"
            
            if col_name not in df.columns:
                df[col_name] = np.nan
            
            df.at[idx, col_name] = value
        
        # Дополнительные признаки
        for feature_name, func in custom_functions.items():
            value = func(glcm_4d)[0]
            col_name = f"{feature_name}_{params_suffix}_{angle_suffix}"
            
            if col_name not in df.columns:
                df[col_name] = np.nan
            
            df.at[idx, col_name] = value
    
    return df

In [19]:
# Для каждого GLCM столбца вызываем функцию
columns_to_process = [
    ('GLCM_a_0', 'G_16_d_2', '0'),
    ('GLCM_a_45', 'G_16_d_2', '45'),
    ('GLCM_a_90', 'G_16_d_2', '90'),
    ('GLCM_a_135', 'G_16_d_2', '135'),
    ('GLCM_b_0', 'G_128_d_4', '0'),
    ('GLCM_b_45', 'G_128_d_4', '45'),
    ('GLCM_b_90', 'G_128_d_4', '90'),
    ('GLCM_b_135', 'G_128_d_4', '135')
]

for glcm_col, params, angle in columns_to_process:
    pneumonia_df = extract_haralick_features_from_column(pneumonia_df, glcm_col, params, angle)
    normal_df = extract_haralick_features_from_column(normal_df, glcm_col, params, angle)

In [20]:
# Переместим целевую переменную в конец вправо
pneumonia_df = pneumonia_df[[col for col in pneumonia_df if col != 'label'] + ['label']]
normal_df = normal_df[[col for col in normal_df if col != 'label'] + ['label']]

Посмотрим структуру получившихся датафреймов

In [21]:
pneumonia_df.columns.tolist()

['GLCM_a_0',
 'GLCM_a_45',
 'GLCM_a_90',
 'GLCM_a_135',
 'GLCM_b_0',
 'GLCM_b_45',
 'GLCM_b_90',
 'GLCM_b_135',
 'contrast_G_16_d_2_0',
 'dissimilarity_G_16_d_2_0',
 'homogeneity_G_16_d_2_0',
 'energy_G_16_d_2_0',
 'correlation_G_16_d_2_0',
 'ASM_G_16_d_2_0',
 'maximum_probability_G_16_d_2_0',
 'entropy_G_16_d_2_0',
 'variance_G_16_d_2_0',
 'sum_average_G_16_d_2_0',
 'sum_variance_G_16_d_2_0',
 'sum_entropy_G_16_d_2_0',
 'difference_variance_G_16_d_2_0',
 'difference_entropy_G_16_d_2_0',
 'contrast_G_16_d_2_45',
 'dissimilarity_G_16_d_2_45',
 'homogeneity_G_16_d_2_45',
 'energy_G_16_d_2_45',
 'correlation_G_16_d_2_45',
 'ASM_G_16_d_2_45',
 'maximum_probability_G_16_d_2_45',
 'entropy_G_16_d_2_45',
 'variance_G_16_d_2_45',
 'sum_average_G_16_d_2_45',
 'sum_variance_G_16_d_2_45',
 'sum_entropy_G_16_d_2_45',
 'difference_variance_G_16_d_2_45',
 'difference_entropy_G_16_d_2_45',
 'contrast_G_16_d_2_90',
 'dissimilarity_G_16_d_2_90',
 'homogeneity_G_16_d_2_90',
 'energy_G_16_d_2_90',
 'corr

In [22]:
normal_df.columns.tolist()

['GLCM_a_0',
 'GLCM_a_45',
 'GLCM_a_90',
 'GLCM_a_135',
 'GLCM_b_0',
 'GLCM_b_45',
 'GLCM_b_90',
 'GLCM_b_135',
 'contrast_G_16_d_2_0',
 'dissimilarity_G_16_d_2_0',
 'homogeneity_G_16_d_2_0',
 'energy_G_16_d_2_0',
 'correlation_G_16_d_2_0',
 'ASM_G_16_d_2_0',
 'maximum_probability_G_16_d_2_0',
 'entropy_G_16_d_2_0',
 'variance_G_16_d_2_0',
 'sum_average_G_16_d_2_0',
 'sum_variance_G_16_d_2_0',
 'sum_entropy_G_16_d_2_0',
 'difference_variance_G_16_d_2_0',
 'difference_entropy_G_16_d_2_0',
 'contrast_G_16_d_2_45',
 'dissimilarity_G_16_d_2_45',
 'homogeneity_G_16_d_2_45',
 'energy_G_16_d_2_45',
 'correlation_G_16_d_2_45',
 'ASM_G_16_d_2_45',
 'maximum_probability_G_16_d_2_45',
 'entropy_G_16_d_2_45',
 'variance_G_16_d_2_45',
 'sum_average_G_16_d_2_45',
 'sum_variance_G_16_d_2_45',
 'sum_entropy_G_16_d_2_45',
 'difference_variance_G_16_d_2_45',
 'difference_entropy_G_16_d_2_45',
 'contrast_G_16_d_2_90',
 'dissimilarity_G_16_d_2_90',
 'homogeneity_G_16_d_2_90',
 'energy_G_16_d_2_90',
 'corr

Оставим только нужные для работы колонки

In [23]:
yes_df = pneumonia_df.drop(columns=['GLCM_a_0', 'GLCM_a_45', 'GLCM_a_90', 'GLCM_a_135', 'GLCM_b_0', 'GLCM_b_45', 'GLCM_b_90', 'GLCM_b_135'])

In [24]:
no_df = normal_df.drop(columns=['GLCM_a_0', 'GLCM_a_45', 'GLCM_a_90', 'GLCM_a_135', 'GLCM_b_0', 'GLCM_b_45', 'GLCM_b_90', 'GLCM_b_135'])

In [25]:
no_df.head()

,contrast_G_16_d_2_0,dissimilarity_G_16_d_2_0,homogeneity_G_16_d_2_0,energy_G_16_d_2_0,correlation_G_16_d_2_0,ASM_G_16_d_2_0,maximum_probability_G_16_d_2_0,entropy_G_16_d_2_0,variance_G_16_d_2_0,sum_average_G_16_d_2_0,...,ASM_G_128_d_4_135,maximum_probability_G_128_d_4_135,entropy_G_128_d_4_135,variance_G_128_d_4_135,sum_average_G_128_d_4_135,sum_variance_G_128_d_4_135,sum_entropy_G_128_d_4_135,difference_variance_G_128_d_4_135,difference_entropy_G_128_d_4_135,label
0,0.234524,0.202501,0.900504,0.262492,0.989218,0.068902,0.136363,4.292329,10.875531,11.364512,...,0.013227,0.110531,9.505176,815.761610,102.461280,3247.413113,7.069563,10.155957,2.907390,0
1,0.222387,0.175280,0.914147,0.266411,0.989235,0.070975,0.144004,4.263930,10.328731,10.327019,...,0.016172,0.123823,9.547434,776.378482,93.423881,3084.795367,7.105421,15.267804,2.883941,0
2,0.262782,0.172143,0.918859,0.253265,0.989225,0.064143,0.100804,4.341063,12.193489,10.972084,...,0.005900,0.056862,9.601174,902.852807,99.202829,3589.696538,7.341396,17.147698,2.749888,0
3,0.314544,0.272949,0.865690,0.234140,0.989809,0.054821,0.103897,4.679440,15.432389,16.885036,...,0.001979,0.023627,10.302352,1095.855019,150.293395,4363.890003,7.571161,11.638070,3.092944,0
4,0.311743,0.272868,0.865826,0.220810,0.986645,0.048757,0.079769,4.760830,11.671177,15.383133,...,0.000917,0.008166,10.622376,842.233769,137.218636,3349.371498,7.679938,10.555597,3.150233,0


In [26]:
yes_df.head()

,contrast_G_16_d_2_0,dissimilarity_G_16_d_2_0,homogeneity_G_16_d_2_0,energy_G_16_d_2_0,correlation_G_16_d_2_0,ASM_G_16_d_2_0,maximum_probability_G_16_d_2_0,entropy_G_16_d_2_0,variance_G_16_d_2_0,sum_average_G_16_d_2_0,...,ASM_G_128_d_4_135,maximum_probability_G_128_d_4_135,entropy_G_128_d_4_135,variance_G_128_d_4_135,sum_average_G_128_d_4_135,sum_variance_G_128_d_4_135,sum_entropy_G_128_d_4_135,difference_variance_G_128_d_4_135,difference_entropy_G_128_d_4_135,label
0,0.517237,0.229644,0.895864,0.266025,0.978052,0.070769,0.147727,4.409857,11.783444,14.458780,...,0.003353,0.045486,10.057317,866.364033,129.209581,3407.775276,7.410094,48.677374,3.072030,1
1,0.236506,0.170149,0.917571,0.288145,0.991254,0.083028,0.137716,4.127582,13.521278,13.823156,...,0.010350,0.094513,9.434395,1008.382471,123.376763,4014.974314,7.114444,13.896080,2.773782,1
2,0.279717,0.260693,0.871084,0.252407,0.981562,0.063709,0.113624,4.334300,7.585415,8.345794,...,0.002634,0.032188,9.915199,553.309731,77.209440,2198.061239,7.284571,8.374911,3.038812,1
3,0.232010,0.186783,0.909624,0.279080,0.988826,0.077886,0.148657,4.261318,10.382118,14.909016,...,0.001863,0.008015,9.912744,748.163331,133.364648,2974.382433,7.404899,12.698546,2.865133,1
4,0.281573,0.175774,0.916492,0.279877,0.980951,0.078331,0.125921,4.133320,7.390807,12.349871,...,0.001538,0.005086,9.919746,526.087039,111.490518,2077.121525,7.356368,21.265066,2.856618,1


In [27]:
no_df.to_csv('NO.csv', index=False, sep=',', encoding='utf-8')
yes_df.to_csv('YES.csv', index=False, sep=',', encoding='utf-8')
print("Сохранено: NO.csv и YES.csv")

Сохранено: NO.csv и YES.csv


### II. Классификация и определение лучших моделей

В среде разработки MATLAB существует большой набор классификаторов. Аналогичные можно найти в библиотеке Python scikit-learn. Такой список выдала языковая модель ChatGPT:

1. Logistic Regression — логистическая регрессия для бинарной классификации
2. Linear SVM — линейный метод опорных векторов
3. SVM RBF — SVM с гауссовым (RBF) ядром
4. SVM Polynomial — SVM с полиномиальным ядром
5. SVM Sigmoid — SVM с сигмоидным ядром
6. Decision Tree — дерево решений
7. Random Forest — ансамбль деревьев (случайный лес)
8. Gradient Boosting — градиентный бустинг деревьев
9. AdaBoost — адаптивный бустинг
10. GentleBoost — мягкий вариант AdaBoost
11. LogitBoost — бустинг с логистической функцией потерь
12. RUSBoost — бустинг с балансировкой классов
13. kNN Euclidean — метод k-ближайших соседей (евклидово расстояние)
14. kNN Minkowski — kNN с расстоянием Минковского
15. kNN Cityblock — kNN с манхэттенским расстоянием
16. kNN Chebyshev — kNN с расстоянием Чебышёва
17. kNN Cosine — kNN с косинусной мерой
18. kNN Mahalanobis — kNN с расстоянием Махаланобиса
19. Naive Bayes Gaussian — гауссовский наивный Байес
20. Naive Bayes Multinomial — мультиномиальный наивный Байес
21. Naive Bayes Bernoulli — бернуллиевский наивный Байес
22. Linear Discriminant Analysis (LDA) — линейный дискриминантный анализ
23. Quadratic Discriminant Analysis (QDA) — квадратичный дискриминантный анализ
24. Subspace KNN — ансамбль kNN на подпространствах
25. Subspace Discriminant — ансамбль дискриминантных моделей

На объединённом наборе данных с двумя классами протестируем работу каждой модели, созласно второй части задания, определим 3 лучших по метрикам качества: точность, чувствительность и специфичность (усреднение для трёх вариантов параметра кроссвалидации K = 3, 4, 5.

In [28]:
df_features = pd.concat([no_df, yes_df], ignore_index=True)

#### Подключение из библиотеки scikit-learn всех необходимых классификаторов

In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    RandomForestClassifier, 
    GradientBoostingClassifier, 
    AdaBoostClassifier,
    BaggingClassifier
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis

In [30]:
# 25 классификаторов

classifiers = {
    # Линейные модели
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    
    # SVM с разными ядрами
    "Linear SVM": SVC(kernel='linear', probability=True, random_state=42),
    "SVM RBF": SVC(kernel='rbf', probability=True, random_state=42),
    "SVM Polynomial": SVC(kernel='poly', probability=True, random_state=42),
    "SVM Sigmoid": SVC(kernel='sigmoid', probability=True, random_state=42),
    
    # Деревья и ансамбли
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, random_state=42),
    
    # Бустинг
    "AdaBoost": AdaBoostClassifier(n_estimators=50, random_state=42),
    "GentleBoost": AdaBoostClassifier(
        estimator=DecisionTreeClassifier(max_depth=1), 
        n_estimators=50, 
        random_state=42,
        learning_rate=0.1
    ),
    "LogitBoost": AdaBoostClassifier(
        estimator=DecisionTreeClassifier(max_depth=1),
        n_estimators=50, 
        random_state=42,
        learning_rate=1.0
    ),
    "RUSBoost": RandomForestClassifier(
        n_estimators=100, 
        class_weight='balanced', 
        random_state=42
    ),
    
    # kNN с разными метриками
    "kNN Euclidean": KNeighborsClassifier(metric='euclidean'),
    "kNN Minkowski": KNeighborsClassifier(metric='minkowski'),
    "kNN Cityblock": KNeighborsClassifier(metric='manhattan'),
    "kNN Chebyshev": KNeighborsClassifier(metric='chebyshev'),
    "kNN Cosine": KNeighborsClassifier(metric='cosine'),
    
    # Наивный Байес
    "Naive Bayes Gaussian": GaussianNB(),
    "Naive Bayes Multinomial": MultinomialNB(),
    "Naive Bayes Bernoulli": BernoulliNB(),
    
    # Дискриминантный анализ
    "Linear Discriminant Analysis (LDA)": LinearDiscriminantAnalysis(),
    "Quadratic Discriminant Analysis (QDA)": QuadraticDiscriminantAnalysis(),
    
    # Subspace методы
    "Subspace KNN": BaggingClassifier(
        estimator=KNeighborsClassifier(), 
        n_estimators=10, 
        random_state=42
    ),
    "Subspace Discriminant": BaggingClassifier(
        estimator=LinearDiscriminantAnalysis(), 
        n_estimators=10, 
        random_state=42
    ),
}

#### Разделение данных на признаки и целевую переменную

In [31]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix
from sklearn.preprocessing import StandardScaler

In [32]:
df_features.head()

,contrast_G_16_d_2_0,dissimilarity_G_16_d_2_0,homogeneity_G_16_d_2_0,energy_G_16_d_2_0,correlation_G_16_d_2_0,ASM_G_16_d_2_0,maximum_probability_G_16_d_2_0,entropy_G_16_d_2_0,variance_G_16_d_2_0,sum_average_G_16_d_2_0,...,ASM_G_128_d_4_135,maximum_probability_G_128_d_4_135,entropy_G_128_d_4_135,variance_G_128_d_4_135,sum_average_G_128_d_4_135,sum_variance_G_128_d_4_135,sum_entropy_G_128_d_4_135,difference_variance_G_128_d_4_135,difference_entropy_G_128_d_4_135,label
0,0.234524,0.202501,0.900504,0.262492,0.989218,0.068902,0.136363,4.292329,10.875531,11.364512,...,0.013227,0.110531,9.505176,815.761610,102.461280,3247.413113,7.069563,10.155957,2.907390,0
1,0.222387,0.175280,0.914147,0.266411,0.989235,0.070975,0.144004,4.263930,10.328731,10.327019,...,0.016172,0.123823,9.547434,776.378482,93.423881,3084.795367,7.105421,15.267804,2.883941,0
2,0.262782,0.172143,0.918859,0.253265,0.989225,0.064143,0.100804,4.341063,12.193489,10.972084,...,0.005900,0.056862,9.601174,902.852807,99.202829,3589.696538,7.341396,17.147698,2.749888,0
3,0.314544,0.272949,0.865690,0.234140,0.989809,0.054821,0.103897,4.679440,15.432389,16.885036,...,0.001979,0.023627,10.302352,1095.855019,150.293395,4363.890003,7.571161,11.638070,3.092944,0
4,0.311743,0.272868,0.865826,0.220810,0.986645,0.048757,0.079769,4.760830,11.671177,15.383133,...,0.000917,0.008166,10.622376,842.233769,137.218636,3349.371498,7.679938,10.555597,3.150233,0


In [33]:
X = df_features.drop(['label'], axis=1)
y = df_features['label']

In [34]:
X.head()

,contrast_G_16_d_2_0,dissimilarity_G_16_d_2_0,homogeneity_G_16_d_2_0,energy_G_16_d_2_0,correlation_G_16_d_2_0,ASM_G_16_d_2_0,maximum_probability_G_16_d_2_0,entropy_G_16_d_2_0,variance_G_16_d_2_0,sum_average_G_16_d_2_0,...,correlation_G_128_d_4_135,ASM_G_128_d_4_135,maximum_probability_G_128_d_4_135,entropy_G_128_d_4_135,variance_G_128_d_4_135,sum_average_G_128_d_4_135,sum_variance_G_128_d_4_135,sum_entropy_G_128_d_4_135,difference_variance_G_128_d_4_135,difference_entropy_G_128_d_4_135
0,0.234524,0.202501,0.900504,0.262492,0.989218,0.068902,0.136363,4.292329,10.875531,11.364512,...,0.990418,0.013227,0.110531,9.505176,815.761610,102.461280,3247.413113,7.069563,10.155957,2.907390
1,0.222387,0.175280,0.914147,0.266411,0.989235,0.070975,0.144004,4.263930,10.328731,10.327019,...,0.986657,0.016172,0.123823,9.547434,776.378482,93.423881,3084.795367,7.105421,15.267804,2.883941
2,0.262782,0.172143,0.918859,0.253265,0.989225,0.064143,0.100804,4.341063,12.193489,10.972084,...,0.987974,0.005900,0.056862,9.601174,902.852807,99.202829,3589.696538,7.341396,17.147698,2.749888
3,0.314544,0.272949,0.865690,0.234140,0.989809,0.054821,0.103897,4.679440,15.432389,16.885036,...,0.991089,0.001979,0.023627,10.302352,1095.855019,150.293395,4363.890003,7.571161,11.638070,3.092944
4,0.311743,0.272868,0.865826,0.220810,0.986645,0.048757,0.079769,4.760830,11.671177,15.383133,...,0.988386,0.000917,0.008166,10.622376,842.233769,137.218636,3349.371498,7.679938,10.555597,3.150233


In [35]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: label, dtype: int64

#### Тестирование классификаторов

In [36]:
K_values = [3, 4, 5]

In [37]:
# Отдельная функция для специфичности
def specificity_score(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    if cm.shape == (2,2):
        tn, fp, fn, tp = cm.ravel()
        return tn / (tn + fp)

In [38]:
# Стандартизация признаков
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [39]:
results = []

print("Начало обучения всех классификаторов...\n")

for name, model in classifiers.items():
    print(f"Обучаем модель: {name}...")
    acc_list, sens_list, spec_list = [], [], []
    try:
        for K in K_values:
            skf = StratifiedKFold(n_splits=K, shuffle=True, random_state=42)
            acc_fold, sens_fold, spec_fold = [], [], []
            for train_idx, test_idx in skf.split(X_scaled, y):
                X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
                y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
                
                # Обучение
                model.fit(X_train, y_train)
                # Предсказание
                y_pred = model.predict(X_test)
                
                acc_fold.append(accuracy_score(y_test, y_pred))
                sens_fold.append(recall_score(y_test, y_pred, average='macro'))
                spec_fold.append(specificity_score(y_test, y_pred))
            
            acc_list.append(np.mean(acc_fold))
            sens_list.append(np.mean(sens_fold))
            spec_list.append(np.mean(spec_fold))
        
        # Усреднение по K
        avg_acc = np.mean(acc_list)
        avg_sens = np.mean(sens_list)
        avg_spec = np.mean(spec_list)
        
        results.append({
            'Model': name,
            'Accuracy': avg_acc,
            'Sensitivity': avg_sens,
            'Specificity': avg_spec
        })
        
        print(f"Результаты {name}: Accuracy={avg_acc:.4f}, Sensitivity={avg_sens:.4f}, Specificity={avg_spec:.4f}\n")

    except Exception as e:
        print(f"Модель {name} пропущена из-за ошибки: {e}\n") # Пропуск классификатора, если возникла ошибка

results_df = pd.DataFrame(results)
results_df.to_csv('results.csv', index=False)
print("Результаты сохранены в 'results.csv'\n")


# Определяем для каждой метрики лучшие модели
top_accuracy = results_df.sort_values(by='Accuracy', ascending=False).head(3)
top_sensitivity = results_df.sort_values(by='Sensitivity', ascending=False).head(3)
top_specificity = results_df.sort_values(by='Specificity', ascending=False).head(3)

# Вывод информации
print("3 лучшие модели по точности (Accuracy):")
for i, row in top_accuracy.iterrows():
    print(f"{row['Model']}: Accuracy = {row['Accuracy']:.4f}")

print("\n3 лучшие модели по чувствительности (Sensitivity):")
for i, row in top_sensitivity.iterrows():
    print(f"{row['Model']}: Sensitivity = {row['Sensitivity']:.4f}")

print("\n3 лучшие модели по специфичности (Specificity):")
for i, row in top_specificity.iterrows():
    print(f"{row['Model']}: Specificity = {row['Specificity']:.4f}")


Начало обучения всех классификаторов...

Обучаем модель: Logistic Regression...
Результаты Logistic Regression: Accuracy=0.8959, Sensitivity=0.8871, Specificity=0.8519

Обучаем модель: Linear SVM...
Результаты Linear SVM: Accuracy=0.9097, Sensitivity=0.9039, Specificity=0.8804

Обучаем модель: SVM RBF...
Результаты SVM RBF: Accuracy=0.8798, Sensitivity=0.8589, Specificity=0.7751

Обучаем модель: SVM Polynomial...
Результаты SVM Polynomial: Accuracy=0.8077, Sensitivity=0.7573, Specificity=0.5557

Обучаем модель: SVM Sigmoid...
Результаты SVM Sigmoid: Accuracy=0.6939, Sensitivity=0.6675, Specificity=0.5624

Обучаем модель: Decision Tree...
Результаты Decision Tree: Accuracy=0.8077, Sensitivity=0.8009, Specificity=0.7736

Обучаем модель: Random Forest...
Результаты Random Forest: Accuracy=0.8798, Sensitivity=0.8636, Specificity=0.7991

Обучаем модель: Gradient Boosting...
Результаты Gradient Boosting: Accuracy=0.8686, Sensitivity=0.8526, Specificity=0.7890

Обучаем модель: AdaBoost...
Рез

Выводы.

Проведена сравнительная оценка 24 классификаторов для диагностики пневмонии на основе текстурных признаков Харалика. 

Линейный дискриминантный анализ (LDA) показал наилучшие результаты по всем метрикам. Subspace Discriminant  и Linear SVM также продемонстрировали высокую эффективность. Наименее успешными оказались SVM с сигмоидальным ядром (69.39%) и полиномиальным ядром (80.77%).
Можно сделать вывод, что для данной задачи классические линейные методы (LDA, Linear SVM) превзошли по результатам качества классификации более сложные нелинейные алгоритмы.
